In [1]:
#Please enter here the paths of the training file and the test file
training_file = "Datasets/seq2tree_geoqueries/train.txt"
test_file = "Datasets/seq2tree_geoqueries/test.txt"
src_vocab_file = "Datasets/seq2tree_geoqueries/vocab.q.txt"
tgt_vocab_file = "Datasets/seq2tree_geoqueries/vocab.f.txt"

"Datasets/seq2tree_geoqueries/vocab.f.txt"

In [1]:
#Please enter here the paths of the training file and the test file
training_file = "Datasets/seq2tree_jobqueries/train.txt"
test_file = "Datasets/seq2tree_jobqueries/test.txt"
src_vocab_file = "Datasets/seq2tree_jobqueries/vocab.q.txt"
tgt_vocab_file = "Datasets/seq2tree_jobqueries/vocab.f.txt"

"Datasets/seq2tree_jobqueries/vocab.f.txt"

In [1]:
#Please enter here the paths of the training file and the test file
training_file = "Datasets/seq2tree_atis/train.txt"
test_file = "Datasets/seq2tree_atis/test.txt"
src_vocab_file = "Datasets/seq2tree_atis/vocab.q.txt"
tgt_vocab_file = "Datasets/seq2tree_atis/vocab.f.txt"

"Datasets/seq2tree_atis/vocab.f.txt"

In [2]:
using Knet: Knet, AutoGrad, param, param0, mat, RNN, relu, Data, adam, progress, nll, zeroone
using Distributions
import .Iterators: cycle, Cycle, take
using IterTools
import CUDA
using CUDA

In [3]:
include("util.jl")

seq2treeAttn

In [4]:
function get_attention_vectors(s::seq2treeAttn, encoder_out, decoder_out, mode::String)
    
    T = size(decoder_out, 3)
    h_enc = permutedims(encoder_out, [3, 1, 2]) # T,H,B
    local attention
   
    h_dec = permutedims(decoder_out[:,:,1:1], [1, 3, 2]) # H,1,B
    dot = Knet.bmm(h_enc, h_dec) # T,1,B
    dot = permutedims(dot, [3, 1, 2]) # B,T,1
    dot = reshape(dot, (size(dot,1), size(dot,2))) # B,T
    attn = Knet.softmax(dot, dims=2) # B,T
    attn = reshape(attn, (size(attn,1), size(attn,2), 1)) # B,T,1
    attn = permutedims(attn, [2, 3, 1]) # T,1,B
    h_enc_tr = permutedims(encoder_out, [1, 3, 2]) # H,T,B
    context = Knet.bmm(h_enc_tr, attn) # H,1,B
    cat_out = cat(h_dec,context; dims=1) # 2H,1,B
    cat_out = permutedims(cat_out, [1, 3, 2]) # 2H,B,1
    cat_out = reshape(cat_out, (size(cat_out,1), size(cat_out,2))) # 2H,B
    h_att = s.linear_att(cat_out) # H,B    
    attention = h_att
    if mode == "test"
        attention = reshape(attention, (size(attention,1), size(attention,2), 1)) # H,B,1
        return attention # H,B,1
    elseif mode == "train"
        if T == 1
            attention = reshape(attention, (size(attention,1), size(attention,2), 1)) # H,B,1
        else
            for i in 2:T
                h_dec = permutedims(decoder_out[:,:,i:i], [1, 3, 2]) # H,1,B
                dot = Knet.bmm(h_enc, h_dec) # T,1,B
                dot = permutedims(dot, [3, 1, 2]) # B,T,1
                dot = reshape(dot, (size(dot,1), size(dot,2))) # B,T
                attn = Knet.softmax(dot, dims=2) # B,T
                attn = reshape(attn, (size(attn,1), size(attn,2), 1)) # B,T,1
                attn = permutedims(attn, [2, 3, 1]) # T,1,B
                h_enc_tr = permutedims(encoder_out, [1, 3, 2]) # H,T,B
                context = Knet.bmm(h_enc_tr, attn) # H,1,B
                cat_out = cat(h_dec,context; dims=1) # 2H,1,B
                cat_out = permutedims(cat_out, [1, 3, 2]) # 2H,B,1
                cat_out = reshape(cat_out, (size(cat_out,1), size(cat_out,2))) # 2H,B
                h_att = s.linear_att(cat_out) # H,B        
                attention = cat(attention, h_att; dims=3)            
            end
        end
        return attention # H,B,T
    end
end

get_attention_vectors (generic function with 1 method)

In [5]:
function (s::seq2treeAttn)(x, x_len, y; average=true) 
    # x: B, T  x_len: B y: B trees
    L = 1
    batchsize = size(x, 1)
    T = size(x, 2)
    @assert length(x_len) == batchsize
    @assert length(x_len) == length(y)
    
    """enc_s = Dict()
    for j in 0:40
        enc_s[j] = Dict()
    end"""
    
    dec_s = Dict()
    for i in 0:120
        dec_s[i] = Dict()
        for j in 0:120
            dec_s[i][j] = Dict()
        end
    end       
    
    """enc_s[0][1] = zeros(Int64, H, batchsize, L)
    enc_s[0][2] = zeros(Int64, H, batchsize, L)"""
    s.encoder.c, s.encoder.h = 0, 0
    x_embedded = s.input_embed(x)
    @assert size(x_embedded)==(X,batchsize,size(x,2)) # X,B,T
    
    """for i in 1:T
        encoder_out = s.encoder(x_embedded[:,:,i]) # H,B
        enc_s[i][1] = s.encoder.c # H,B,L
        enc_s[i][2] = s.encoder.h # H,B,L
    end"""
    
    encoder_out = s.encoder(x_embedded)

    
    #encoder_out = s.encoder(x_embedded[:,:,i]); @assert size(encoder_out)==(H,batchsize,size(x,2)) # H,B,T
      
    queue_tree = Dict()    
    
    for i in 1:batchsize
        queue_tree[i] = Any[]
        push!(queue_tree[i], Dict("tree" => y[i], "parent"=> 0, "child_index"=> 1))
    end
    
    loss = 0
    cur_index, max_index = 1, 1
    dec_batch = Dict()
    dec_batch_masked = Dict()
    
    while (cur_index <= max_index)
        
        max_w_len = -1
        batch_w_list = Any[]
        
        for i in 1:batchsize
            w_list = Any[]
            if (cur_index <= length(queue_tree[i]))
                t = queue_tree[i][cur_index]["tree"]
                for ic in 1:t.num_children
                    if typeof(t.children[ic]) == Tree
                        push!(w_list, tok2int_output["<n>"])
                        push!(queue_tree[i], Dict("tree" => t.children[ic], "parent" => cur_index, "child_index"=> ic)) 
                    else
                        push!(w_list, t.children[ic])
                    end
                end
                if length(queue_tree[i]) > max_index
                    max_index = length(queue_tree[i])
                end
            end
            if length(w_list) > max_w_len
                max_w_len = length(w_list)
            end
            push!(batch_w_list, w_list)
        end
        #dec_batch: max_cur_index, B, Tmaxx +2
        #dec_batch[cur_index] = fill(tok2int_output["</s>"], (batchsize, max_w_len+2)) 
       
        dec_batch[cur_index] = fill(2, (batchsize, max_w_len+2))        
        
        for i in 1:batchsize
            w_list = batch_w_list[i]
            if length(w_list) > 0
                for j in 1:length(w_list)
                    dec_batch[cur_index][i,j+1] = w_list[j]
                end
                
                # add start and end tokens
                if cur_index == 1
                    dec_batch[cur_index][i,1] = tok2int_output["<s>"]
                else
                    dec_batch[cur_index][i,1] = tok2int_output["("]
                end
                dec_batch[cur_index][i,length(w_list) + 2] = tok2int_output["</s>"]
            end
        end
        
        dec_batch_masked[cur_index] = mask(dec_batch[cur_index], 2)
        
        dec_s[cur_index][0][1] = convert(Knet.KnetArray{Float32}, zeros(H, batchsize, 1)) 
        dec_s[cur_index][0][2] = convert(Knet.KnetArray{Float32}, zeros(H, batchsize, 1))
        
        if cur_index == 1
            for i in 1:batchsize
                """dec_s[1][0][1][:,i,:] = Knet.value(enc_s[x_len[i]][1][:,i,:])
                dec_s[1][0][2][:,i,:] = Knet.value(enc_s[x_len[i]][2][:,i,:])"""
                dec_s[1][0][1] = s.encoder.c
                dec_s[1][0][2] = s.encoder.h
            end
        else
            for i in 1:batchsize
                if (cur_index <= length(queue_tree[i]))
                    par_index = queue_tree[i][cur_index]["parent"]
                    child_index = queue_tree[i][cur_index]["child_index"]
                    dec_s[cur_index][0][1][:,i,:] .= Knet.value(dec_s[par_index][child_index][1][:,i,:])
                    dec_s[cur_index][0][2][:,i,:] .= Knet.value(dec_s[par_index][child_index][2][:,i,:])
                end
            end
        end
        
        labels = dec_batch[cur_index][:,2]
        for i in 1:size(dec_batch[cur_index], 2)-1
            s.decoder.c = dec_s[cur_index][i-1][1] # H,B,L
            s.decoder.h = dec_s[cur_index][i-1][2] # H,B,L
            y_embedded = s.output_embed(dec_batch[cur_index][:,i])
            @assert size(y_embedded)==(X,batchsize) # X,B
                        
            decoder_out = s.decoder(y_embedded) # H, B
            
            decoder_out_for_attn = reshape(decoder_out, (size(decoder_out,1), size(decoder_out,2), 1)) # H,B,1
            dec_s[cur_index][i][1] = s.decoder.c # H,B,L
            dec_s[cur_index][i][2] = s.decoder.h # H,B,L
            
            attention = get_attention_vectors(s, encoder_out, decoder_out_for_attn, "train")
            global dims
            dims = size(attention) # H,B,Tmaxx-1
            
            scores = s.linear(Knet.dropout(reshape(attention, dims[1], dims[2] * dims[3]), s.dropout))
            #scores = s.linear(Knet.dropout(decoder_out, s.dropout))
            @assert size(scores)==(Va,batchsize) #Va, 
            
            
            loss += nll(scores, labels; dims=1, average=false)[1]
            
            if i != size(dec_batch[cur_index], 2)-1
                for j in 1:batchsize
                    if Int(dec_batch[cur_index][j,i+1]) == 2
                        labels[j] = 0
                    else
                        labels[j] = Int(dec_batch[cur_index][j,i+2]) 
                    end
                end
            end    
        end
        cur_index += 1
    end
    loss = loss / batchsize
    return loss
end

In [6]:
### FOR PREDICTION/TESTING
function (s::seq2treeAttn)(x_data)
    
    x = copy(x_data) 
    push!(x, tok2int_input["<s>"])
    pushfirst!(x, tok2int_input["</s>"])
    x = reverse(x) # T 
    x = reshape(x, 1, length(x)) # 1,T
    
    x_embedded = s.input_embed(x)
    @assert size(x_embedded)==(X,1,length(x)) # X,1,T 
    
    s.encoder.c, s.encoder.h = 0, 0
    
    encoder_out = s.encoder(x_embedded)
    @assert size(encoder_out)==(H,1,length(x))# H,1,T   
    
    queue_decode = Any[] 
    push!(queue_decode, Dict("ch" => (s.encoder.c, s.encoder.h), "parent"=>0, "child_index"=>1, "t"=> Tree()))
    
    head = 1
    while head <= length(queue_decode) && head <= 100
        
        ch = queue_decode[head]["ch"]
        t = queue_decode[head]["t"]
        
        if head == 1
            prev_word = [tok2int_output["<s>"]] 
        else
            prev_word = [tok2int_output["("]] 
        end
        
        i_child = 1
        while true
            prev_word_embed = s.output_embed(prev_word[1])
            @assert size(prev_word_embed)==(X,) # X
            
            s.decoder.c, s.decoder.h = ch[1], ch[2]
            @assert size(ch[1]) == (H, 1, 1)
            
            decoder_out = s.decoder(prev_word_embed)
            @assert size(decoder_out)==(H,) # H
            
            decoder_out_for_attn = reshape(decoder_out, (size(decoder_out,1), 1, 1)) # H,1,1
            
            attention = get_attention_vectors(s, encoder_out, decoder_out_for_attn, "test") # H,1,1
            dims = size(attention) # H,1,1,
            output = s.linear(reshape(attention, dims[1], dims[2] * dims[3]))
            @assert size(output)==(Va, 1) # Va, 1
            
            ch = (s.decoder.c, s.decoder.h) # H, 1, 1
            @assert size(ch[1]) == (H, 1, 1)
            prev_word = argmax(output) # cartesian index(sth, 1)
            
            if Int(prev_word[1]) == tok2int_output["</s>"] || t.num_children >= 120
                break
            elseif Int(prev_word[1]) == tok2int_output["<n>"]                 
                push!(queue_decode, Dict("ch" => (copy(ch[1]), copy(ch[2])), "parent"=>head, "child_index"=>i_child, "t"=> Tree()))                
                add_child(t, prev_word[1])
            else
                add_child(t, prev_word[1])
            end
            i_child += 1
        end
        
        head += 1
    end
    
    for i in length(queue_decode):-1:2
        cur = queue_decode[i]
        queue_decode[cur["parent"]]["t"].children[cur["child_index"]] = cur["t"] 
    end
    
    return to_list(queue_decode[1]["t"], tok2int_output)
    
end

In [7]:
int2tok_input, tok2int_input, int2tok_output, tok2int_output, Vq, Va = tok_int(src_vocab_file, tgt_vocab_file)
trndata, trn_n_in, trn_n_out = data_reader(training_file, tok2int_input, tok2int_output)
tstdata, tst_n_in, tst_n_out = data_reader(test_file, tok2int_input, tok2int_output)
X = 200
H = 200 
trndata_batch = minibatch2(trndata, 20)

221-element Array{Any,1}:
 ([1 1 … 111 2; 1 1 … 66 2; … ; 1 18 … 58 2; 1 5 … 111 2], Any[3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], Any[Tree(nothing, 4, Any[13, 7, 14, Tree(Tree(#= circular reference @-3 =#), 2, Any[82, 7])]), Tree(nothing, 4, Any[13, 7, 14, Tree(Tree(#= circular reference @-3 =#), 2, Any[48, 7])]), Tree(nothing, 4, Any[13, 7, 14, Tree(Tree(#= circular reference @-3 =#), 3, Any[8, Tree(Tree(#= circular reference @-3 =#), 2, Any[82, 7]), Tree(Tree(#= circular reference @-3 =#), 3, Any[93, 7, 9])])]), Tree(nothing, 4, Any[13, 7, 14, Tree(Tree(#= circular reference @-3 =#), 3, Any[8, Tree(Tree(#= circular reference @-3 =#), 2, Any[41, 7]), Tree(Tree(#= circular reference @-3 =#), 3, Any[37, 7, 9])])]), Tree(nothing, 1, Any[21]), Tree(nothing, 4, Any[13, 7, 14, Tree(Tree(#= circular reference @-3 =#), 3, Any[8, Tree(Tree(#= circular reference @-3 =#), 2, Any[82, 7]), Tree(Tree(#= circular reference @-3 =#), 3, Any[93, 7, 9])])]), Tree(nothing, 4, Any[13, 

In [8]:
lrdecay!(s::seq2treeAttn, decay::Real) =
    for p in Knet.params(s); p.opt.lr = p.opt.lr*decay; end

lrdecay! (generic function with 1 method)

In [9]:
my_model = seq2treeAttn(X,H,Vq,Va,0.3)

seq2treeAttn(Embed(P(Knet.KnetArrays.KnetArray{Float32,2}(200,338))), Embed(P(Knet.KnetArrays.KnetArray{Float32,2}(200,170))), LSTM(input=200,hidden=200,dropout=0.3), LSTM(input=200,hidden=200,dropout=0.3), Linear(P(Knet.KnetArrays.KnetArray{Float32,2}(170,200)), P(Knet.KnetArrays.KnetArray{Float32,1}(170)), identity), Linear(P(Knet.KnetArrays.KnetArray{Float32,2}(200,400)), P(Knet.KnetArrays.KnetArray{Float32,1}(200)), tanh), 0.3)

In [10]:
Knet.progress!(Knet.rmsprop(my_model, ncycle(trndata_batch, 4); rho=0.95, gclip=5, lr = 0.005))
println("train accuracy: ", model_accuracy(my_model, trndata))

for i in 5:180
    
    #println(Knet.params(my_model)[1].opt.lr)
    Knet.rmsprop!(my_model, trndata_batch;rho=0.95,gclip=5)
    lrdecay!(my_model, 0.97)
    if i == 20
        for p in Knet.params(my_model); p.opt.lr = 0.005; end
    end
    
    if i%10 == 0
        println("After $i epochs")
        println("train accuracy: ", model_accuracy(my_model, trndata))
        println("test accuracy: ", model_accuracy(my_model, tstdata))
    end
end


┣███████████████████▉┫ [100.00%, 884/884, 01:53/01:53, 7.79i/s] 


train accuracy: 0.009217625899280575
After 10 epochs
train accuracy: 0.1164568345323741
test accuracy: 0.125
After 20 epochs
train accuracy: 0.4662769784172662
test accuracy: 0.38169642857142855
After 30 epochs
train accuracy: 0.5328237410071942
test accuracy: 0.43973214285714285
After 40 epochs
train accuracy: 0.6726618705035972
test accuracy: 0.5669642857142857
After 50 epochs
train accuracy: 0.8642086330935251
test accuracy: 0.671875
After 60 epochs
train accuracy: 0.9251348920863309
test accuracy: 0.7142857142857143
After 70 epochs
train accuracy: 0.9593075539568345
test accuracy: 0.7254464285714286
After 80 epochs
train accuracy: 0.9831384892086331
test accuracy: 0.7366071428571429
After 90 epochs
train accuracy: 0.9842625899280576
test accuracy: 0.7366071428571429
After 100 epochs
train accuracy: 0.9847122302158273
test accuracy: 0.75
After 110 epochs
train accuracy: 0.985386690647482
test accuracy: 0.7455357142857143
After 120 epochs
train accuracy: 0.9860611510791367
test accur